In [1]:
from dataclasses import dataclass
import numpy as np
from deck import full_euchre_deck
from play_round import PlayRound
from branch_calc import (
    filter_branch_by_hand,
    setdiff2d_idx,
    suit_id,
    find_winner,
    calc_all_possible_hands,
)
from dealer import Dealer

In [2]:
game = Dealer(deck=full_euchre_deck ,players=4)
game.deal_cards()
hands5 = [game.hand1, game.hand2, game.hand3, game.hand4]
hands5
# first_play = [game.hand1[0], game.hand1[1], game.hand1[2], game.hand1[3], game.hand1[4]]

[array([[  12,    0],
        [  14,    0],
        [   0,  100],
        [  10,    0],
        [   0, -135]]),
 array([[  0,  90],
        [  9,   0],
        [  0, 130],
        [-11,   0],
        [ -9,   0]]),
 array([[ 13,   0],
        [  0, -13],
        [-13,   0],
        [-10,   0],
        [  0,  -9]]),
 array([[  0, 140],
        [  0, -12],
        [-14,   0],
        [  0, 120],
        [  0, -11]])]

In [21]:
score = []

In [ ]:

for i in range(125):
    try:
        round1 = PlayRound(hands=hands5, winner=0, card_play=0)
        round1.play_round(bit=i)
        score.append(round1.round_winning_team)
    except IndexError :
        print('All possible tricks played')
        print(score)
        break

All possible trick played
[1, 0, 0, 1, 0]
